In [1]:
import pandas as pd
import numpy as np
import values
import uuid
import random
from datetime import datetime, timedelta

##### Tables

In [2]:
# CDHDR
cdhdr_cols = ['MANDT', 'OBJECTCLAS', 'OBJECTID', 'CHANGENR', 'USERNAME', 'ERDAT']
cdhdr = pd.DataFrame(columns=cdhdr_cols)

# CDPOS
cdpos_cols = ['MANDT', 'CHANGENR', 'TABNAME', 'FNAME', 'OLDVALUE', 'NEWVALUE']
cdpos = pd.DataFrame(columns=cdpos_cols)

# Materials
mara_cols = ['MANDT', 'MATNR', 'MAKTX']
mara = pd.DataFrame(columns=mara_cols)

# Users
usr01_cols = ['MANDT', 'BNAME', 'UTYPE']
usr01 = pd.DataFrame(columns=usr01_cols)

# Customers
Kna1_cols = ['MANDT', 'KUNNR', 'NAME1', 'ERDAT', 'ERNAM']
kna1 = pd.DataFrame(columns=Kna1_cols)

# Sales Order header
vbak_cols = ['MANDT', 'VBELN', 'ERDAT', 'ERNAM', 'KUNNR', 'NETWR', 'VSBED', 'FAKSK']
# vbak_cols = ['MANDT', 'VBELN', 'ERDAT', 'ERNAM', 'KUNNR', 'NETWR', 'shp. cond.', 'bil. blk.']
vbak = pd.DataFrame(columns=vbak_cols)

# Sales Order items
vbap_cols = ['MANDT', 'VBELN', 'POSNR', 'KWMENG', 'MATNR', 'NETWR']
vbap =pd.DataFrame(columns=vbap_cols)

# Delivery header
likp_cols = ['MANDT', 'VBELN', 'WADAT', 'LFUHR', 'ERDAT', 'ERNAM']
likp = pd.DataFrame(columns=likp_cols)

# Delivery items
lips_cols = ['MANDT', 'VBELN', 'POSNR']
lips = pd.DataFrame(columns=lips_cols)

# Header status
vbuk_cols = ['MANDT', 'VBELN', 'LFSTK', 'FKSTK', 'AEDAT', 'VBTYP']
# vbuk_cols = ['MANDT', 'VBELN', 'del. stat', 'bil. stat', 'changed on', 'sd cat.']
vbuk = pd.DataFrame(columns=vbuk_cols)

# Material movement header
mkpf_cols = ['MANDT', 'MBLNR', 'BLART', 'AEDAT', 'USNAM']
mkpf = pd.DataFrame(columns=mkpf_cols)

# Material movement items
mseg_cols = ['MANDT', 'MBLNR', 'KDAUF', 'KDPOS']
mseg = pd.DataFrame(columns=mseg_cols)

# Billing header
vbrk_cols = ['MANDT', 'VBELN', 'VBTYP', 'ERNAM', 'ERDAT']
vbrk  =pd.DataFrame(columns=vbrk_cols)

# Billing items
vbrp_cols = ['MANDT', 'VBELN', 'POSNR']
vbrp  =pd.DataFrame(columns=vbrp_cols)

##### Helpers

In [3]:
UPTO_YEAR = lambda: timedelta(days=random.randint(1, 365))
UPTO_MONTH = lambda: timedelta(days=random.randint(1, 30))
UPTO_WEEK = lambda: timedelta(days=random.randint(1, 7))
UPTO_DAY = lambda: timedelta(days=0, hours=random.randint(0, 23))
UPTO_HOUR = lambda: timedelta(days=0, hours=0, minutes=random.randint(0, 59))

In [4]:
vbak_len = 100

In [5]:
def generate_random_datetime(start_date, end_date):
    time_delta = end_date - start_date
    random_days = random.randint(0, time_delta.days)
    random_seconds = random.randint(0, 86400)  # 86400 seconds in a day

    random_datetime = start_date + timedelta(days=random_days, seconds=random_seconds)
    return random_datetime

def divide_num_in_normal(num: float, num_parts: int):
    mean = num/2
    std_dev = num/15

    random_numbers = np.random.normal(mean, std_dev, num_parts)
    random_numbers /= sum(random_numbers)

    result = [int(number * num) for number in random_numbers]
    result[-1] += num - sum(result)

    return result

In [6]:
# Materials
mara_temp_list = []
for i in range(100):
    rand_id = uuid.uuid4()
    mara_temp_list.append([values.mandt, rand_id, values.material_names[i]])

mara = pd.concat([mara, pd.DataFrame(mara_temp_list, columns=mara_cols)], ignore_index=True)

In [7]:
# Users
usr01_temp_list = []
for u_name, u_type in values.users.items():
    usr01_temp_list.append([values.mandt, u_name, u_type])

usr01 = pd.concat([usr01, pd.DataFrame(usr01_temp_list, columns=usr01_cols)], ignore_index=True)

In [8]:
# Customers
kna1_temp_list = []
for company in values.company_names:
    rand_id = uuid.uuid4()
    rand_dt = values.times['customer_create_start'] + UPTO_YEAR()
    rand_usr = random.choice(list(values.users.keys()))
    kna1_temp_list.append([values.mandt, rand_id, company, rand_dt, rand_usr])

kna1 = pd.concat([kna1, pd.DataFrame(kna1_temp_list, columns=Kna1_cols)], ignore_index=True)

In [9]:
vbak_temp_list = []
vbap_temp_list = []
likp_temp_list = []
lips_temp_list = []
cdhdr_changes_temp_list = []
cdpos_changes_temp_list = []
vbuk_temp_list = []
mkpf_temp_list = []
mseg_temp_list = []
vbrk_temp_list = []
vbrp_temp_list = []

In [10]:
class SalesOrderItem:
    def __init__(self, vbeln, posnr, kwmeng, matnr, netwr) -> None:
        self.mandt = values.mandt
        self.vbeln = vbeln
        self.posnr = posnr
        self.kwmeng = kwmeng
        self.matnr = matnr
        self.netwr = netwr
        
class SalesOrder:
    def __init__(self) -> None:
        global vbap_temp_list
        global likp_temp_list
        global lips_temp_list
        global cdhdr_changes_temp_list
        global cdpos_changes_temp_list
        global mkpf_temp_list
        global mseg_temp_list
        global vbrk_temp_list
        global vbrp_temp_list
        global kna1

        self.mandt = values.mandt
        self.vbeln = uuid.uuid4()
        rand_kna1 = kna1.sample(n=1)
        self.erdat = generate_random_datetime(datetime.fromtimestamp(rand_kna1['ERDAT'].values[0].tolist() / 1e9), values.times['salesorder_create_end'])
        self.ernam = random.choice(list(values.users.keys()))
        self.kunnr = rand_kna1['KUNNR'].values[0]
        self.netwr = round(random.uniform(500, 15_000), 2)
        self.vsbed = random.choice(values.shipping_conditions)
        self.faksk = pd.NA

        self.vbaps = []
        self.vbuk = self.vbuk = [values.mandt, self.vbeln, 'SALESORDERCREATED', pd.NA, self.erdat, 'C']

        self.delivery_doc_created_at = None
        self.planned_delivery_dt = None
        self.actual_delivery_dt = None
        self.delivery_released_at = None
        self.goods_shipped_at = None
        self.delivery_confirmed_at = None
        self.invoice_sent_at = None
        self.invoice_cleared_at = None
        self.billing_block_set_at = None
        self.billing_block_removed_at = None
        self.sd_cancelled_at = None
        self.sd_returned_at = None
        self.latest_activity_at = self.erdat

        self.has_billing_block = True if random.uniform(0, 1) <= 0.3 else False
        self.has_remove_delivery_block = True if self.has_billing_block and (random.uniform(0, 1) <= 0.9) else False
        self.has_cancel_order = True if random.uniform(0, 1) <= 0.3 else False
        self.has_return_order = True if random.uniform(0, 1) <= 0.3 else False
        
        # Create Sales order 
        self.create_vbap_rows(num_items=(random.randint(5, 15)))

    def update_latest_activity_time(self, new_latest_time):
        if self.latest_activity_at < new_latest_time:
            self.latest_activity_at = new_latest_time


    def create_vbap_rows(self, num_items):
        # Activity: Create sales order - item
        vbap_netwrs = divide_num_in_normal(num=self.netwr, num_parts=num_items)
        
        for j in range(num_items):
            rand_matnr = mara.sample(n=1)['MATNR'].values[0]
            quantity = random.randint(50, 150)
            vbap_temp_list.append([values.mandt, self.vbeln, j, quantity, rand_matnr, vbap_netwrs[j]])
            self.vbaps.append(SalesOrderItem(self.vbeln, j, quantity, rand_matnr, vbap_netwrs[j]))

    def generate_delivery_doc(self):
        # Activity: Generate delivery document
        self.delivery_doc_created_at = self.latest_activity_at + UPTO_MONTH()
        self.update_latest_activity_time(new_latest_time=self.delivery_doc_created_at)

        self.planned_delivery_dt = self.delivery_doc_created_at + UPTO_MONTH()
        self.actual_delivery_dt = self.planned_delivery_dt if random.uniform(0, 1) > 0.25 else self.planned_delivery_dt + UPTO_WEEK()
        rand_usr = random.choice(list(values.users.keys()))
        likp_temp_list.append([values.mandt, self.vbeln, self.planned_delivery_dt, self.actual_delivery_dt, self.delivery_doc_created_at, rand_usr])
        
        self.generate_delivery_doc_item()
    
    def generate_delivery_doc_item(self):
        # Activity: Generate delivery document - item
        for vbap_elem in self.vbaps:
            lips_temp_list.append([values.mandt, self.vbeln, vbap_elem.posnr])

    def release_delivery(self):
        # Activity: Release delivery
        rand_changenr = uuid.uuid4()
        rand_usr = random.choice(list(values.users.keys()))
        self.delivery_released_at = self.latest_activity_at + UPTO_MONTH() if self.delivery_doc_created_at == None else self.delivery_doc_created_at + UPTO_WEEK()
        self.update_latest_activity_time(new_latest_time=self.delivery_released_at)

        # record change
        cdhdr_changes_temp_list.append([values.mandt, 'DELIVERY', self.vbeln, rand_changenr, rand_usr, self.delivery_released_at])
        cdpos_changes_temp_list.append([values.mandt, rand_changenr, 'VBUK', 'LFSTK', pd.NA, 'DELIVERYRELEASED'])
        
        self.vbuk = [values.mandt, self.vbeln, 'DELIVERYRELEASED', pd.NA, self.delivery_released_at, 'C']

    def ship_goods(self):
        # Activity: Ship goods
        rand_mblnr = uuid.uuid4()
        self.goods_shipped_at = self.latest_activity_at + UPTO_MONTH() if self.delivery_released_at == None else self.delivery_released_at + UPTO_WEEK()
        self.update_latest_activity_time(new_latest_time=self.goods_shipped_at)
        rand_usr = random.choice(list(values.users.keys()))

        mkpf_temp_list.append([values.mandt, rand_mblnr, 'GOODSISSUE', self.goods_shipped_at, rand_usr])

        for vbap_elem in self.vbaps:
            mseg_temp_list.append([values.mandt, rand_mblnr, self.vbeln, vbap_elem.posnr])

    def send_invoice(self):
        # Activity: Send invoice
        self.invoice_sent_at =  self.latest_activity_at + UPTO_MONTH() if self.goods_shipped_at == None else self.goods_shipped_at + UPTO_HOUR()
        self.update_latest_activity_time(new_latest_time=self.invoice_sent_at)
        rand_usr = random.choice(list(values.users.keys()))

        vbrk_temp_list.append([values.mandt, self.vbeln, 'INVOICE', rand_usr, self.invoice_sent_at])
        
        self.send_invoice_items()

    def send_invoice_items(self):
        for vbap_elem in self.vbaps:
            vbrp_temp_list.append([values.mandt, self.vbeln, vbap_elem.posnr])
    
    def receive_delivery_confirmation(self):
        # Activity: Recieve delivery confirmation
        rand_changenr = uuid.uuid4()
        rand_usr = random.choice(list(values.users.keys()))

        self.delivery_confirmed_at = self.latest_activity_at + UPTO_MONTH() if self.actual_delivery_dt == None else self.actual_delivery_dt + UPTO_DAY()
        self.update_latest_activity_time(new_latest_time=self.delivery_confirmed_at)
        self.vbuk = [values.mandt, self.vbeln, 'DELIVERYCONFIRMED', pd.NA, self.delivery_confirmed_at, 'C']

        # record change
        cdhdr_changes_temp_list.append([values.mandt, 'DELIVERY', self.vbeln, rand_changenr, rand_usr, self.delivery_confirmed_at])
        cdpos_changes_temp_list.append([values.mandt, rand_changenr, 'VBUK', 'LFSTK', 'DELIVERYRELEASED', 'DELIVERYCONFIRMED'])

    def clear_invoice(self):
        # Activity: Clear invoice
        rand_changenr = uuid.uuid4()
        rand_usr = random.choice(list(values.users.keys()))

        self.invoice_cleared_at = self.latest_activity_at + UPTO_YEAR()
        self.update_latest_activity_time(new_latest_time=self.invoice_cleared_at)

        self.vbuk = [values.mandt, self.vbeln, 'DELIVERYCONFIRMED', 'INVOICECLEARED', self.invoice_cleared_at, 'C']

        # record change
        cdhdr_changes_temp_list.append([values.mandt, 'BILLING', self.vbeln, rand_changenr, rand_usr, self.invoice_cleared_at])
        cdpos_changes_temp_list.append([values.mandt, rand_changenr, 'VBUK', 'FKSTK', pd.NA, 'INVOICECLEARED'])


In [11]:
class BillingDeviation:
    def __init__(self, vbak: SalesOrder) -> None:
        self.vbak = vbak
        global cdhdr_changes_temp_list
        global cdpos_changes_temp_list
        
    def set_billing_block(self):
        # Activity: Set billing block
        self.vbak.faksk = 'BILLINGBLOCK'

        rand_changenr = uuid.uuid4()
        rand_usr = random.choice(list(values.users.keys()))

        self.vbak.billing_block_set_at = self.vbak.latest_activity_at + UPTO_MONTH() if self.vbak.delivery_released_at == None else self.vbak.delivery_released_at + UPTO_DAY()
        self.vbak.update_latest_activity_time(new_latest_time=self.vbak.billing_block_set_at)

        cdhdr_changes_temp_list.append([values.mandt, 'BILLING', self.vbak.vbeln, rand_changenr, rand_usr, self.vbak.billing_block_set_at])
        cdpos_changes_temp_list.append([values.mandt, rand_changenr, 'VBAK', 'FAKSK', pd.NA, 'BILLINGBLOCK'])

    def remove_billing_block(self):
        # Activity: Remove billing block
        self.vbak.faksk = pd.NA

        rand_changenr = uuid.uuid4()
        rand_usr = random.choice(list(values.users.keys()))

        self.vbak.billing_block_removed_at = self.vbak.latest_activity_at + UPTO_WEEK() if self.vbak.goods_shipped_at == None else self.vbak.goods_shipped_at + UPTO_WEEK()
        self.vbak.update_latest_activity_time(new_latest_time=self.vbak.billing_block_removed_at)

        cdhdr_changes_temp_list.append([values.mandt, 'BILLING', self.vbak.vbeln, rand_changenr, rand_usr, self.vbak.billing_block_removed_at])
        cdpos_changes_temp_list.append([values.mandt, rand_changenr, 'VBAK', 'FAKSK', 'BILLINGBLOCK', pd.NA])

In [12]:
class CancelOrReturn:
    def __init__(self, vbak) -> None:
        self.vbak = vbak
    
    def cancel_order(self):
        # Activity: Cancel order
        rand_changenr = uuid.uuid4()
        rand_usr = random.choice(list(values.users.keys()))

        self.vbak.sd_cancelled_at = self.vbak.latest_activity_at + UPTO_DAY()
        self.vbak.update_latest_activity_time(new_latest_time=self.vbak.sd_cancelled_at)

        cdhdr_changes_temp_list.append([values.mandt, 'SALESORDER', self.vbak.vbeln, rand_changenr, rand_usr, self.vbak.sd_cancelled_at])
        cdpos_changes_temp_list.append([values.mandt, rand_changenr, 'VBUK', 'VBTYP', self.vbak.vbuk[-1], 'h'])
        self.vbak.vbuk[-1] = 'h'

    def return_goods(self):
        # Activity: Cancel order
        rand_changenr = uuid.uuid4()
        rand_usr = random.choice(list(values.users.keys()))

        self.vbak.sd_returned_at = self.vbak.latest_activity_at + UPTO_MONTH()
        self.vbak.update_latest_activity_time(new_latest_time=self.vbak.sd_returned_at)

        cdhdr_changes_temp_list.append([values.mandt, 'SALESORDER', self.vbak.vbeln, rand_changenr, rand_usr, self.vbak.sd_returned_at])
        cdpos_changes_temp_list.append([values.mandt, rand_changenr, 'VBUK', 'VBTYP', self.vbak.vbuk[-1], 'H'])
        self.vbak.vbuk[-1] = 'H'

In [13]:
transition_matrix = np.array([
       [0.01, 0.90, 0.04, 0.02, 0.00, 0.00, 0.00, 0.00, 0.00, 0.02, 0.01], # generate delivery doc
       [0.00, 0.01, 0.03, 0.90, 0.00, 0.06, 0.00, 0.00, 0.00, 0.00, 0.00], # release delivery
       [0.00, 0.00, 0.00, 0.85, 0.05, 0.00, 0.10, 0.00, 0.00, 0.00, 0.00], # set billing block
       [0.00, 0.00, 0.00, 0.05, 0.05, 0.78, 0.03, 0.03, 0.01, 0.05, 0.00], # ship goods
       [0.00, 0.00, 0.00, 0.00, 0.05, 0.85, 0.05, 0.01, 0.03, 0.01, 0.00], # remove billing block
       [0.00, 0.00, 0.00, 0.00, 0.00, 0.05, 0.05, 0.80, 0.05, 0.05, 0.00], # send invoice
       [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 1.00], # cancel order
       [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.05, 0.10, 0.80, 0.05], # receive delivery confirmation
       [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 1.00], # return goods
       [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.05, 0.95],  # clear invoice
       [0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 1.00]  # end
])

for i in range(1_000):
    vbak_obj = SalesOrder()
    billing_dev_obj = BillingDeviation(vbak=vbak_obj)
    cancel_return_dev_obj = CancelOrReturn(vbak=vbak_obj)

    activities = [
        vbak_obj.generate_delivery_doc, 
        vbak_obj.release_delivery, 
        billing_dev_obj.set_billing_block, 
        vbak_obj.ship_goods, 
        billing_dev_obj.remove_billing_block, 
        vbak_obj.send_invoice, 
        cancel_return_dev_obj.cancel_order, 
        vbak_obj.receive_delivery_confirmation, 
        cancel_return_dev_obj.return_goods, 
        vbak_obj.clear_invoice,
        'end'
    ]

    step = 0
    while step < len(activities) - 1:
        activities[step]()
        step = np.random.choice(len(activities), p=transition_matrix[step])
        
        if step == len(activities) - 1:
            break

    vbuk_temp_list.append(vbak_obj.vbuk)
    vbak_temp_list.append([values.mandt, vbak_obj.vbeln, vbak_obj.erdat, vbak_obj.ernam, vbak_obj.kunnr, vbak_obj.netwr, vbak_obj.vsbed, vbak_obj.faksk])

In [14]:
vbak = pd.concat([vbak, pd.DataFrame(vbak_temp_list, columns=vbak_cols)], ignore_index=True)
vbap = pd.concat([vbap, pd.DataFrame(vbap_temp_list, columns=vbap_cols)], ignore_index=True)
likp = pd.concat([likp, pd.DataFrame(likp_temp_list, columns=likp_cols)], ignore_index=True)
lips = pd.concat([lips, pd.DataFrame(lips_temp_list, columns=lips_cols)], ignore_index=True)
cdhdr = pd.concat([cdhdr, pd.DataFrame(cdhdr_changes_temp_list, columns=cdhdr_cols)], ignore_index=True)
cdpos = pd.concat([cdpos, pd.DataFrame(cdpos_changes_temp_list, columns=cdpos_cols)], ignore_index=True)
vbuk = pd.concat([vbuk, pd.DataFrame(vbuk_temp_list, columns=vbuk_cols)], ignore_index=True)
mkpf = pd.concat([mkpf, pd.DataFrame(mkpf_temp_list, columns=mkpf_cols)], ignore_index=True)
mseg = pd.concat([mseg, pd.DataFrame(mseg_temp_list, columns=mseg_cols)], ignore_index=True)
vbrk = pd.concat([vbrk, pd.DataFrame(vbrk_temp_list, columns=vbrk_cols)], ignore_index=True)
vbrp = pd.concat([vbrp, pd.DataFrame(vbrp_temp_list, columns=vbrp_cols)], ignore_index=True)

##### Save csv

In [15]:
cdhdr.to_csv(f'data/cdhdr.csv', index=False) 
cdpos.to_csv(f'data/cdpos.csv', index=False) 
mara.to_csv(f'data/mara.csv', index=False) 
usr01.to_csv(f'data/usr01.csv', index=False) 
kna1.to_csv(f'data/kna1.csv', index=False) 
vbak.to_csv(f'data/vbak.csv', index=False) 
vbap.to_csv(f'data/vbap.csv', index=False) 
likp.to_csv(f'data/likp.csv', index=False) 
lips.to_csv(f'data/lips.csv', index=False) 
vbuk.to_csv(f'data/vbuk.csv', index=False) 
mkpf.to_csv(f'data/mkpf.csv', index=False) 
mseg.to_csv(f'data/mseg.csv', index=False) 
vbrk .to_csv(f'data/vbrk.csv', index=False) 
vbrp .to_csv(f'data/vbrp.csv', index=False) 

In [16]:
vbap

,MANDT,VBELN,POSNR,KWMENG,MATNR,NETWR
0,SC1,fa1a1720-4d44-45ce-ad25-19d3c45ec7ae,0,124,cb88dd04-f603-48be-bb72-1694d5f72557,964.00
1,SC1,fa1a1720-4d44-45ce-ad25-19d3c45ec7ae,1,115,597d8a25-db62-4cd4-8f4a-1dd3bc0118c9,696.00
2,SC1,fa1a1720-4d44-45ce-ad25-19d3c45ec7ae,2,93,e6f95bee-ab5b-4909-86e3-19f5837e532e,1029.00
3,SC1,fa1a1720-4d44-45ce-ad25-19d3c45ec7ae,3,98,3553448f-7034-420a-b0c5-8de8f3508926,783.00
4,SC1,fa1a1720-4d44-45ce-ad25-19d3c45ec7ae,4,150,d7c05583-e5b6-4001-b94b-b02bc72f4f88,793.00
...,...,...,...,...,...,...
9990,SC1,2538db64-7e7d-4100-a638-04d08d5fc2b1,2,50,a817e67f-132a-4832-92eb-c3cd9445f66f,1810.00
9991,SC1,2538db64-7e7d-4100-a638-04d08d5fc2b1,3,123,2bc20ed6-62eb-412c-b70f-582ca8aefbef,1768.00
9992,SC1,2538db64-7e7d-4100-a638-04d08d5fc2b1,4,124,b75c1166-fffd-4997-acd0-e5ee10596a73,1843.00
9993,SC1,2538db64-7e7d-4100-a638-04d08d5fc2b1,5,85,a493f3da-75c6-442d-8960-ff209ba7fb83,1365.00
